# Лабораторная работа 1, студент Устинов Денис Александрович М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для задачи классификации

Задача: Определение типа активности человека на основе данных с носимых устройств

Набор данных: Human Activity Recognition Dataset (HAR) (https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones), где данные получены с носимых датчиков и классифицируются в такие категории, как ходьба, сидение, подъем по лестнице и т. д.

Обоснование выбора: HAR представляет практическую задачу для классификации с несколькими классами и используется в задачах мониторинга здоровья и физической активности. Задача распознавания активности с помощью KNN полезна и практична, так как такой алгоритм достаточно прост для реализации в системах с носимыми устройствами.

### b. Набор данных для задачи регрессии

Задача: Прогнозирование уровня выбросов CO2 автомобилей

Набор данных: CO2 Emission by Vehicles (https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles), включающий параметры автомобиля (тип двигателя, вес, объем двигателя) и уровень выбросов CO2.

Обоснование выбора: Прогнозирование выбросов позволяет оценивать экологическое воздействие автомобилей на основе их характеристик, что является важной задачей в экологии и инженерии. Такой набор данных также подходит для KNN, так как близость характеристик автомобилей позволяет строить точные прогнозы выбросов.

### c. Выбор метрик качества

Для задачи классификации:

1) Accuracy (Точность): доля правильно классифицированных объектов. Хорошо подходит для задачи с равномерным распределением классов.
2) Precision: можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
3) Recall: показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
2) F1-score: среднее гармоническое между точностью (precision) и полнотой (recall), особенно полезно, если классы несбалансированы, так как учитывает ложноположительные и ложноотрицательные предсказания.

Для задачи регрессии:

1) Mean Absolute Error (MAE): средняя абсолютная ошибка, показывает среднее отклонение предсказаний от фактических значений и хорошо интерпретируется.
2) Mean Squared Error (MSE): средняя квадратичная ошибка, показывает более крупные ошибки больше чем MAE, что делает его полезным для выявления больших отклонений в прогнозах.

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

Импорт библиотек

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


#### Предобработаем данные

Получение и очистка данных

In [28]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

Дубликаты:

In [29]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

Подготовим обучающую выборку

In [30]:
def map_func(arg):
    if arg == 1: 
        return 'WALKING'
    if arg == 2:
        return 'WALKING_UPSTAIRS'
    if arg == 3:
        return 'WALKING_DOWNSTAIRS'
    if arg == 4:
        return 'SITTING'
    if arg == 5:
        return 'STANDING'
    if arg == 6:
        return 'LAYING'

In [31]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])
y_train_labels = y_train.map(map_func)

train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_11889/3107987138.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
5334,0.275424,-0.014081,-0.105677,-0.997051,-0.980195,-0.972942,-0.997176,-0.979383,-0.971368,-0.942773,...,0.102889,-0.192463,0.266583,-0.205337,-0.900241,0.025413,0.01962,25,4,SITTING


Подготовим тестовую выборку

In [32]:
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None)

y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])
y_test_labels = y_test.map(map_func)

test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_11889/2096605161.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
1800,0.288717,-0.02206,-0.092009,-0.37185,0.205228,-0.411859,-0.434948,0.128024,-0.384601,-0.049755,...,-0.316344,0.239978,0.378916,-0.537225,-0.713396,0.295043,-0.003574,13,1,WALKING


Поменяем названия функций

In [33]:
columns = train.columns

columns = columns.str.replace('[()]','')
columns = columns.str.replace('[-]', '')
columns = columns.str.replace('[,]','')

train.columns = columns
test.columns = columns

test.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

Сохраним выборки как csv

In [34]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

#### Обучим модель KNN

Функция чтения датасета

In [35]:
def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

Функция обучения KNN модели

In [36]:
def train_model(train_x, train_y, validation=None):
    model = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None)
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}")       

Загрузим данные

In [37]:
train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

Классификация

In [38]:
model = train_model(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.9002375296912114
Recall = 0.9002375296912114
Precision = 0.9039585713237325
F1 Score = 0.8993037837295996


### b. Обучить модели из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)']+ [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn_regressor = KNeighborsRegressor(n_neighbors=5, weights='uniform', p=2, metric='manhattan')
knn_regressor.fit(X_train, y_train)

y_pred = knn_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE (CO2 Emissions): {mae:.2f}")
print(f"MSE (CO2 Emissions): {mse:.2f}")

MAE (CO2 Emissions): 3.93
MSE (CO2 Emissions): 76.53


## 3. Улучшение бейзлайна

### a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.
2. Подбор гиперпараметров

    - Оптимизация количества соседей (n_neighbors): Ключевой гиперпараметр для KNN, который можно подобрать с использованием кросс-валидации. Также можно рассмотреть более адаптивные способы выбора соседей, например, весовые функции.
    - Выбор метрики расстояния: Проверить влияние различных метрик (евклидова, манхэттенского расстояния и т.д.) на результат классификации и регрессии.
    - Кросс-валидация: Проведение кросс-валидации с подбором гиперпараметров для нахождения оптимального n_neighbors, weights, metric и других гиперпараметров.

3. Методы оптимизации
    - Grid Search и Random Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели и Random Search для расширения возможностей поиска с более низкой вычислительной затратой.

### Обучение моделей, оценка качества обучения моделей по метрикам

#### Подбор гиперпараметров и Grid Search

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Гиперпараметры для классификации
param_grid_class = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_class = KNeighborsClassifier()
grid_search_class = GridSearchCV(knn_class, param_grid_class, cv=5, scoring='accuracy')
grid_search_class.fit(train_X, train_y)
best_knn_class = grid_search_class.best_estimator_

print("Лучшие параметры для классификации (HAR):", grid_search_class.best_params_)
print("Лучший результат на кросс-валидации (HAR):", grid_search_class.best_score_)

y_pred_class = best_knn_class.predict(test_X)
accuracy = accuracy_score(test_y, y_pred_class)
precision = precision_score(test_y, y_pred_class, average='weighted')
recall = recall_score(test_y, y_pred_class, average='weighted')
f1 = f1_score(test_y, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Гиперпараметры для регрессии
param_grid_reg = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_reg = KNeighborsRegressor()
grid_search_reg = GridSearchCV(knn_reg, param_grid_reg, cv=5, scoring='neg_mean_squared_error')
grid_search_reg.fit(X_train, y_train)
best_knn_reg = grid_search_reg.best_estimator_

print("Лучшие параметры для регрессии (CO2):", grid_search_reg.best_params_)
print("Лучший результат на кросс-валидации (CO2):", -grid_search_reg.best_score_)

y_pred_reg = best_knn_reg.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_reg))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_reg))

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/User

Лучшие параметры для классификации (HAR): {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Лучший результат на кросс-валидации (HAR): 0.9816374625989077
Accuracy: 0.9049881235154394
Precision: 0.9083027317222394
Recall: 0.9049881235154394
F1 Score: 0.9040905399655608
Лучшие параметры для регрессии (CO2): {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Лучший результат на кросс-валидации (CO2): 45.26808079174798
MAE (CO2 Emissions): 2.3889615412605654
MSE (CO2 Emissions): 54.58617459908717


#### Random Search

In [41]:
from sklearn.model_selection import RandomizedSearchCV

random_search_class = RandomizedSearchCV(knn_class, param_grid_class, n_iter=10, cv=5, scoring='accuracy', random_state=0)
random_search_class.fit(train_X, train_y)
best_knn_class = random_search_class.best_estimator_
print("Лучшие параметры для классификации (HAR) с Random Search:", random_search_class.best_params_)
print("Лучший результат на кросс-валидации (HAR) с Random Search:", random_search_class.best_score_)

y_pred_class = best_knn_class.predict(test_X)
accuracy = accuracy_score(test_y, y_pred_class)
precision = precision_score(test_y, y_pred_class, average='weighted')
recall = recall_score(test_y, y_pred_class, average='weighted')
f1 = f1_score(test_y, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

random_search_reg = RandomizedSearchCV(knn_reg, param_grid_reg, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=0)
random_search_reg.fit(X_train, y_train)
best_knn_reg = random_search_reg.best_estimator_
print("Лучшие параметры для регрессии (CO2) с Random Search:", random_search_reg.best_params_)
print("Лучший результат на кросс-валидации (CO2) с Random Search:", -random_search_reg.best_score_)

y_pred_reg = best_knn_reg.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_reg))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_reg))

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/User

Лучшие параметры для классификации (HAR) с Random Search: {'weights': 'distance', 'n_neighbors': 5, 'metric': 'manhattan'}
Лучший результат на кросс-валидации (HAR) с Random Search: 0.9785095057737575
Accuracy: 0.9134713267729895
Precision: 0.9171879851153816
Recall: 0.9134713267729895
F1 Score: 0.9126698790513678
Лучшие параметры для регрессии (CO2) с Random Search: {'weights': 'distance', 'n_neighbors': 3, 'metric': 'euclidean'}
Лучший результат на кросс-валидации (CO2) с Random Search: 46.201229542998945
MAE (CO2 Emissions): 2.395207429291693
MSE (CO2 Emissions): 48.30084075689549


#### Добавление нового признака (для задачи регрессии)

In [42]:
from sklearn.preprocessing import PolynomialFeatures

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

knn = KNeighborsRegressor()
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_poly, y_train_new_field)

best_knn = grid_search.best_estimator_

y_pred = best_knn.predict(X_test_poly)

mae = mean_absolute_error(y_test_new_field, y_pred)
mse = mean_squared_error(y_test_new_field, y_pred)

print("Лучшие гиперпараметры:", grid_search.best_params_)
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

Лучшие гиперпараметры: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
MAE: 2.24
MSE: 34.31


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'weights': 'distance', 'n_neighbors': 5, 'metric': 'manhattan'}

In [43]:
def train_model(train_x, train_y, validation=None):
    model = KNeighborsClassifier(n_neighbors=5, weights='distance', metric='manhattan')
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}") 

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')      

model = train_model(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.9134713267729895
Recall = 0.9134713267729895
Precision = 0.9171879851153816
F1 Score = 0.9126698790513678


#### Задача регрессии

Лучшие параметры для регрессии: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

knn_regressor = KNeighborsRegressor(n_neighbors=3, weights='distance', p=1)
knn_regressor.fit(X_train_poly, y_train_new_field)

y_pred = knn_regressor.predict(X_test_poly)

mae = mean_absolute_error(y_test_new_field, y_pred)
mse = mean_squared_error(y_test_new_field, y_pred)

print(f"MAE (CO2 Emissions): {mae:.2f}")
print(f"MSE (CO2 Emissions): {mse:.2f}")

MAE (CO2 Emissions): 2.24
MSE (CO2 Emissions): 34.31


### Выводы

В задаче классификации на наборе данных HAR изначальная модель KNN показала хорошие результаты, однако оптимизация с помощью Grid Search позволила улучшить точность и F1-меру. Применение Манхэттенского расстояния и уменьшение числа соседей до трёх показали, что правильная настройка гиперпараметров помогает модели точнее классифицировать активности.

В задаче регрессии для прогноза выбросов CO2 изначальные значения метрик (MAE и MSE) были выше, чем хотелось бы, но подбор оптимальных параметров также позволил улучшить результаты. Grid Search помог найти оптимальные значения, которые снизили среднюю абсолютную ошибку и среднеквадратичное отклонение, что подтверждает важность выбора подходящей метрики расстояния и числа соседей для KNN-регрессии.

Дополнительное улучшение удалось достичь с помощью добавления нового признака Engine_Fuel, представляющего произведение объёма двигателя и расхода топлива. Этот признак позволил лучше учесть зависимость между техническими характеристиками и выбросами, что привело к снижению ошибок регрессии. Таким образом, гипотеза о том, что добавление информативных признаков может улучшить модель, подтвердилась.

В результате, комбинируя подбор гиперпараметров и создание новых признаков, удалось существенно повысить качество моделей, что подчёркивает значимость этих методов в машинном обучении.

## 4. Имплементация алгоритма машинного обучения 

### Имплементация KNN для классификации

In [2]:
class KNNClassifier:
    def __init__(self, n_neighbors=5, weights='uniform', metric='minkowski', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _compute_distance(self, x):
        if self.metric == 'minkowski':
            return np.linalg.norm(self.X_train - x, ord=self.p, axis=1)
        elif self.metric == 'euclidean':
            return np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Invalid metric parameter")

    def _get_weights(self, distances):
        if self.weights == 'uniform':
            return np.ones_like(distances)
        elif self.weights == 'distance':
            return 1 / (distances + 1e-5)
        else:
            raise ValueError("Invalid weights parameter")

    def predict(self, X):
        predictions = []
        for x in X:
            distances = self._compute_distance(x)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_labels = self.y_train[nearest_indices]
            nearest_distances = distances[nearest_indices]
            weights = self._get_weights(nearest_distances)
            weighted_votes = {}
            
            for label, weight in zip(nearest_labels, weights):
                weighted_votes[label] = weighted_votes.get(label, 0) + weight
            
            prediction = max(weighted_votes, key=weighted_votes.get)
            predictions.append(prediction)
        return np.array(predictions)

### Имплементация KNN для регрессии

In [3]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, weights='uniform', metric='minkowski', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.p = p

    def fit(self, X, y):
        self.X_train = np.array(X, dtype=float)
        self.y_train = np.array(y, dtype=float)

    def _compute_distance(self, x):
        x = np.array(x, dtype=float)
        if self.metric == 'minkowski':
            return np.linalg.norm(self.X_train - x, ord=self.p, axis=1)
        elif self.metric == 'euclidean':
            return np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Invalid metric parameter")

    def _get_weights(self, distances):
        if self.weights == 'uniform':
            return np.ones_like(distances)
        elif self.weights == 'distance':
            return 1 / (distances + 1e-5)
        else:
            raise ValueError("Invalid weights parameter")

    def predict(self, X):
        X = np.array(X, dtype=float)
        predictions = []
        for x in X:
            distances = self._compute_distance(x)
            nearest_indices = np.argsort(distances)[:self.n_neighbors]
            nearest_targets = self.y_train[nearest_indices]
            nearest_distances = distances[nearest_indices]
            weights = self._get_weights(nearest_distances)
            
            weighted_sum = np.sum(weights * nearest_targets)
            weighted_mean = weighted_sum / np.sum(weights)
            predictions.append(weighted_mean)
        return np.array(predictions)

### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

knn_classifier = KNNClassifier(n_neighbors=5, metric='minkowski', weights='distance')
knn_classifier.fit(train_X, train_y)
predictions = knn_classifier.predict(test_X)

accuracy = accuracy_score(test_y, predictions)
precision = precision_score(test_y, predictions, average='weighted')
recall = recall_score(test_y, predictions, average='weighted')
f1 = f1_score(test_y, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.9002375296912114
Precision: 0.9039585713237325
Recall: 0.9002375296912114
F1 Score: 0.8993037837295996


#### Задача регрессии

In [20]:
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

train_X_reg = np.array(X_train)
train_y_reg = np.array(y_train)
test_X_reg = np.array(X_test)
test_y_reg = np.array(y_test)

knn_regressor = KNNRegressor(n_neighbors=5, metric='manhattan', weights='uniform')
knn_regressor.fit(train_X_reg, train_y_reg)

predictions = knn_regressor.predict(test_X_reg)
mae = np.mean(np.abs(predictions - test_y_reg))
mse = np.mean((predictions - test_y_reg) ** 2)

print(f"MAE: {mae}")
print(f"MSE: {mse}")

MAE: 3.500631768953068
MSE: 76.68792418772563


### Сравнение результатов с п.2. Выводы

Для классификации метрики библиотечного и самописного алгоритма KNN практически идентичны, что говорит о корректности реализации собственной версии классификатора. Точное совпадение показателя Accuracy и близкие значения для Precision, Recall, и F1-score свидетельствуют о том, что самописный KNN классификатор правильно повторяет работу стандартного из sklearn. 

Для задачи регрессии самописная модель KNN показывает метрики, близкие к результатам sklearn. Значения MAE и MSE немного ниже, чем значения в алгоритме из sklearn.

Таких результаты могут говорить об отсутствии каких-либо оптимизаций в самостоятельных имплементаций или небольшими различиями в начальных параметрах обучения модели.

Таким образом, библиотечные и самописные версии KNN дают схожие результаты, что свидетельствует о корректности реализации и эффективности самописных алгоритмов. Незначительные улучшения в регрессионных метриках, достигнутые самописной моделью, подтверждают, что алгоритм правильно обрабатывает данные и адекватно решает задачу предсказания.

### Улучшение бейзлайна

#### Подбор гиперпараметров и Grid Search

In [21]:
from itertools import product

best_params_classification = {}
best_metrics_classification = {"accuracy": 0}

best_params_regression = {}
best_metrics_regression = {"mse": float("inf")}

n_neighbors_options = [3, 5, 7]
metric_options = ['euclidean', 'manhattan']
weights_options = ['uniform', 'distance']

for n_neighbors, metric, weights in product(n_neighbors_options, metric_options, weights_options):
    custom_knn_clf = KNNClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
    custom_knn_clf.fit(train_X, train_y)
    predictions = custom_knn_clf.predict(test_X)

    accuracy = accuracy_score(test_y, predictions)
    precision = precision_score(test_y, predictions, average='weighted')
    recall = recall_score(test_y, predictions, average='weighted')
    f1 = f1_score(test_y, predictions, average='weighted')

    if accuracy > best_metrics_classification["accuracy"]:
        best_params_classification = {
            "n_neighbors": n_neighbors,
            "metric": metric,
            "weights": weights
        }
        best_metrics_classification = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        }

for n_neighbors, metric, weights in product(n_neighbors_options, metric_options, weights_options):
    custom_knn_reg = KNNRegressor(n_neighbors=n_neighbors, metric=metric, weights=weights)
    custom_knn_reg.fit(train_X_reg, train_y_reg)
    predictions = custom_knn_reg.predict(test_X_reg)

    mae = mean_absolute_error(test_y_reg, predictions)
    mse = mean_squared_error(test_y_reg, predictions)

    if mse < best_metrics_regression["mse"]:
        best_params_regression = {
            "n_neighbors": n_neighbors,
            "metric": metric,
            "weights": weights
        }
        best_metrics_regression = {
            "mae": mae,
            "mse": mse
        }

print("Лучшие параметры для классификации:", best_params_classification)
print("Метрики для классификации:", best_metrics_classification)
print("Лучшие параметры для регрессии:", best_params_regression)
print("Метрики для регрессии:", best_metrics_regression)

Лучшие параметры для классификации: {'n_neighbors': 5, 'metric': 'manhattan', 'weights': 'uniform'}
Метрики для классификации: {'accuracy': 0.9141499830335935, 'precision': np.float64(0.9177927943138419), 'recall': np.float64(0.9141499830335935), 'f1': np.float64(0.9133600056382576)}
Лучшие параметры для регрессии: {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'distance'}
Метрики для регрессии: {'mae': np.float64(2.3511805033647586), 'mse': np.float64(47.90677007651014)}


#### Random Search

In [22]:
import random

def random_search(n_iterations, model_type="classification"):
    best_params = {}
    best_metrics = {"accuracy": 0, "precision" : 0, "recall" : 0, "f1" : 0} if model_type == "classification" else {"mae" : float("inf"), "mse": float("inf")}
    
    for _ in range(n_iterations):
        n_neighbors = random.choice(n_neighbors_options)
        metric = random.choice(metric_options)
        weights = random.choice(weights_options)
        
        if model_type == "classification":
            model = KNNClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
            model.fit(train_X, train_y)
            predictions = model.predict(test_X)
            
            accuracy = accuracy_score(test_y, predictions)
            precision = precision_score(test_y, predictions, average='weighted')
            recall = recall_score(test_y, predictions, average='weighted')
            f1 = f1_score(test_y, predictions, average='weighted')
            if accuracy > best_metrics["accuracy"]:
                best_params = {"n_neighbors": n_neighbors, "metric": metric, "weights": weights}
                best_metrics = {
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1": f1
                }
                
        else:
            model = KNNRegressor(n_neighbors=n_neighbors, metric=metric, weights=weights)
            model.fit(train_X_reg, train_y_reg)
            predictions = model.predict(test_X_reg)
            
            mae = mean_absolute_error(test_y_reg, predictions)
            mse = mean_squared_error(test_y_reg, predictions)
            if mse < best_metrics["mse"]:
                best_params = {"n_neighbors": n_neighbors, "metric": metric, "weights": weights}
                best_metrics = {
                    "mae": mae,
                    "mse": mse
                }
    
    return best_params, best_metrics

best_params_classification_random, best_metrics_classification_random = random_search(20, model_type="classification")
best_params_regression_random, best_metrics_regression_random = random_search(20, model_type="regression")

print("Лучшие параметры для классификации (Random Search):", best_params_classification_random)
print("Метрики для классификации (Random Search):", best_metrics_classification_random)
print("Лучшие параметры для регрессии (Random Search):", best_params_regression_random)
print("Метрики для регрессии (Random Search):", best_metrics_regression_random)

Лучшие параметры для классификации (Random Search): {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'uniform'}
Метрики для классификации (Random Search): {'accuracy': 0.9141499830335935, 'precision': np.float64(0.918065595424), 'recall': np.float64(0.9141499830335935), 'f1': np.float64(0.9132612560503154)}
Лучшие параметры для регрессии (Random Search): {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'distance'}
Метрики для регрессии (Random Search): {'mae': np.float64(2.3511805033647586), 'mse': np.float64(47.90677007651014)}


#### Добавление нового признака (для задачи регрессии)

In [23]:
X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

train_X_reg = X_train_new_field.values
train_y_reg = y_train_new_field.values
test_X_reg = X_test_new_field.values
test_y_reg = y_test_new_field.values

best_regressor = KNNRegressor(**best_params_regression)
best_regressor.fit(train_X_reg, train_y_reg)
predictions = best_regressor.predict(test_X_reg)

mae_new = mean_absolute_error(test_y_reg, predictions)
mse_new = mean_squared_error(test_y_reg, predictions)

print("Метрики регрессии после добавления нового признака:")
print(f"MAE: {mae_new}")
print(f"MSE: {mse_new}")

Метрики регрессии после добавления нового признака:
MAE: 2.3240175516796655
MSE: 32.90820790923369


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'n_neighbors': 5, 'metric': 'manhattan', 'weights': 'uniform'}

In [16]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

knn_classifier = KNNClassifier(n_neighbors=5, metric='manhattan', weights='uniform')
knn_classifier.fit(train_X, train_y)
predictions = knn_classifier.predict(test_X)

accuracy = accuracy_score(test_y, predictions)
precision = precision_score(test_y, predictions, average='weighted')
recall = recall_score(test_y, predictions, average='weighted')
f1 = f1_score(test_y, predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.9141499830335935
Precision: 0.9177927943138419
Recall: 0.9141499830335935
F1 Score: 0.9133600056382576


#### Задача регрессии

Лучшие параметры для регрессии: {'n_neighbors': 7, 'metric': 'manhattan', 'weights': 'distance'}

In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

# Добавление нового признака "Engine_Fuel" как "Engine Size" * "Fuel Consumption Comb"
X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

train_X_reg = X_train_new_field.values
train_y_reg = y_train_new_field.values
test_X_reg = X_test_new_field.values
test_y_reg = y_test_new_field.values

best_regressor = KNNRegressor(n_neighbors=7, metric='manhattan', weights='distance')
best_regressor.fit(train_X_reg, train_y_reg)
predictions = best_regressor.predict(test_X_reg)

mae_new = mean_absolute_error(test_y_reg, predictions)
mse_new = mean_squared_error(test_y_reg, predictions)

print("Метрики регрессии:")
print(f"MAE: {mae_new}")
print(f"MSE: {mse_new}")

Метрики регрессии:
MAE: 2.3240175516796655
MSE: 32.90820790923369


#### Выводы

В результате применения методов улучшения бейзлайна (Grid Search, Random Search и добавление нового признака) для как моделей из sklearn, так и самописных, удалось достичь значительных улучшений по метрикам точности, ошибок и других показателей. 

Наилучшие результаты были получены после применения техники подбора гиперпараметров с Grid Search и добавления нового признака, что позволило улучшить метрики как для задач классификации, так и для регрессии.

Подбор гиперпараметров с помощью Grid Search и Random Search привел к улучшению метрик классификации и регрессии. Например, после подбора гипермпараметров и Grid Search для классификации на задаче HAR точность составила 0.914, а для регрессии на CO2 MAE был 2.39. Добавление нового признака для регрессии значительно улучшило метрики (MAE = 2.24, MSE = 34.31).

Результаты с самописными моделями показали аналогичные улучшения. Например, для классификации точность составила 0.914, а для регрессии MAE был 2.35 и MSE 47.91. Добавление нового признака также улучшило метрики (MAE = 2.32, MSE = 32.91).

# Лабораторная работа 2, студент Устинов Денис Александрович М8О-406Б-21

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели логистической регрессии из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pandas as pd
import numpy as np
import sklearn.utils

def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

def train_model(train_x, train_y, validation=None):
    model = LogisticRegression()
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}")
    
    return model

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

model = train_model(train_X, train_y, validation=(test_X, test_y))


Accuracy = 0.9586019681031558
Recall = 0.9586019681031558
Precision = 0.9597941694255984
F1 Score = 0.9584674380821284


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### b. Обучить модели из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

y_pred = linear_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE (CO2 Emissions): {mae:.2f}")
print(f"MSE (CO2 Emissions): {mse:.2f}")


MAE (CO2 Emissions): 13.13
MSE (CO2 Emissions): 377.79


## 3. Улучшение бейзлайна

### a. Сформулировать гипотезы (препроцессинг данных, визуализация данных, формирование новых признаков, подбор гиперпараметров на кросс-валидации и т.д.)

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - Для логистической регрессии:

        - C (инверсия регуляризации): Найти оптимальный уровень регуляризации. Маленький C — сильная регуляризация, большой C — слабая.
        - solver: Подобрать лучший метод оптимизации (например, 'lbfgs', 'liblinear', 'saga' и т.д.) для конкретных данных.

    - Для линейной регрессии:

        - alpha: Параметр регуляризации для методов Ridge и Lasso.
        - Проверить влияние применения регуляризованных моделей:
            - Ridge (L2-регуляризация).
            - Lasso (L1-регуляризация).

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

### Обучение моделей, оценка качества обучения моделей по метрикам

#### Масштабируем данные

In [ ]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

#### Подбор гиперпараметров и Grid Search

#### Логистическая регрессия (классификация)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

param_grid_logreg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [1000, 5000, 10000],
    'solver': ['lbfgs', 'liblinear'],
}

logreg = LogisticRegression(solver='lbfgs')
grid_search_logreg = GridSearchCV(logreg, param_grid_logreg, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_logreg.fit(train_X_scaled, train_y)
best_logreg = grid_search_logreg.best_estimator_

print("Лучшие параметры для логистической регрессии (HAR):", grid_search_logreg.best_params_)
print("Лучший результат на кросс-валидации (HAR):", grid_search_logreg.best_score_)

y_pred_logreg = best_logreg.predict(test_X_scaled)
accuracy = accuracy_score(test_y, y_pred_logreg)
precision = precision_score(test_y, y_pred_logreg, average='weighted')
recall = recall_score(test_y, y_pred_logreg, average='weighted')
f1 = f1_score(test_y, y_pred_logreg, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Лучшие параметры для логистической регрессии (HAR): {'C': 1, 'max_iter': 1000, 'solver': 'liblinear'}
Лучший результат на кросс-валидации (HAR): 0.9849009188991709
Accuracy: 0.9613165931455717
Precision: 0.963249239599378
Recall: 0.9613165931455717
F1 Score: 0.9613709750318309


#### Линейная регрессия (регрессия)

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

param_grid_ridge = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga']
}

ridge = Ridge()
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ridge.fit(X_train, y_train)
best_ridge = grid_search_ridge.best_estimator_

print("Лучшие параметры для Ridge регрессии (CO2):", grid_search_ridge.best_params_)
print("Лучший результат на кросс-валидации (CO2):", -grid_search_ridge.best_score_)

y_pred_ridge = best_ridge.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_ridge))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_ridge))

param_grid_lasso = {
    'alpha': [0.01, 0.1, 1, 10, 100],
    'max_iter': [1000, 5000, 10000]
}

lasso = Lasso()
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lasso.fit(X_train, y_train)
best_lasso = grid_search_lasso.best_estimator_

print("Лучшие параметры для Lasso регрессии (CO2):", grid_search_lasso.best_params_)
print("Лучший результат на кросс-валидации (CO2):", -grid_search_lasso.best_score_)

y_pred_lasso = best_lasso.predict(X_test)
print("MAE (CO2 Emissions):", mean_absolute_error(y_test, y_pred_lasso))
print("MSE (CO2 Emissions):", mean_squared_error(y_test, y_pred_lasso))


Лучшие параметры для Ridge регрессии (CO2): {'alpha': 10, 'solver': 'lsqr'}
Лучший результат на кросс-валидации (CO2): 375.35785425391526
MAE (CO2 Emissions): 13.12377206913496
MSE (CO2 Emissions): 380.5580689476625


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.332e+04, tolerance: 1.443e+03
  model = cd_fast.enet_coordinate_descent(


Лучшие параметры для Lasso регрессии (CO2): {'alpha': 0.01, 'max_iter': 1000}
Лучший результат на кросс-валидации (CO2): 375.7968494885642
MAE (CO2 Emissions): 13.146135034120853
MSE (CO2 Emissions): 378.50093303746394


#### Добавление нового признака (для задачи регрессии)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

ridge = Ridge(max_iter=20000)
param_grid_ridge = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'saga']
}
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_ridge.fit(X_train_poly_scaled, y_train_new_field)
best_ridge = grid_search_ridge.best_estimator_

y_pred_ridge = best_ridge.predict(X_test_poly_scaled)
ridge_mae = mean_absolute_error(y_test_new_field, y_pred_ridge)
ridge_mse = mean_squared_error(y_test_new_field, y_pred_ridge)

print("Лучшие параметры для Ridge:", grid_search_ridge.best_params_)
print(f"Ridge MAE: {ridge_mae:.2f}")
print(f"Ridge MSE: {ridge_mse:.2f}")

lasso = Lasso(max_iter=20000)
param_grid_lasso = {
    'alpha': [0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [20000]
}
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lasso.fit(X_train_poly_scaled, y_train_new_field)
best_lasso = grid_search_lasso.best_estimator_

y_pred_lasso = best_lasso.predict(X_test_poly_scaled)
lasso_mae = mean_absolute_error(y_test_new_field, y_pred_lasso)
lasso_mse = mean_squared_error(y_test_new_field, y_pred_lasso)

print("Лучшие параметры для Lasso:", grid_search_lasso.best_params_)
print(f"Lasso MAE: {lasso_mae:.2f}")
print(f"Lasso MSE: {lasso_mse:.2f}")


Лучшие параметры для Ridge: {'alpha': 0.1, 'solver': 'svd'}
Ridge MAE: 7.29
Ridge MSE: 139.02


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.221e+03, tolerance: 1.421e+03
  model = cd_fast.enet_coordinate_descent(
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.627e+03, tolerance: 1.434e+03
  model = cd_fast.enet_coordinate_descent(
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

Лучшие параметры для Lasso: {'alpha': 0.01, 'max_iter': 20000}
Lasso MAE: 7.45
Lasso MSE: 147.72


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.905e+03, tolerance: 1.779e+03
  model = cd_fast.enet_coordinate_descent(


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации:
{'C': 1, 'max_iter': 1000, 'solver': 'liblinear'}

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pandas as pd
import numpy as np
import sklearn.utils

def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

def train_model(train_x, train_y, validation=None):
    model = LogisticRegression(max_iter=1000, solver='liblinear', multi_class='ovr', C = 1)
    
    model.fit(train_x, train_y)
    
    if validation is not None:
        y_hat = model.predict(validation[0])
        acc = metrics.accuracy_score(validation[1], y_hat)
        print(f"Accuracy = {acc}")
        recall = metrics.recall_score(validation[1], y_hat, average='weighted')
        precision = metrics.precision_score(validation[1], y_hat, average='weighted')
        f1 = metrics.f1_score(validation[1], y_hat, average='weighted')
        print(f"Recall = {recall}")
        print(f"Precision = {precision}")
        print(f"F1 Score = {f1}")
    
    return model

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

model = train_model(train_X, train_y, validation=(test_X, test_y))

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy = 0.9619952494061758
Recall = 0.9619952494061758
Precision = 0.9632384391448742
F1 Score = 0.9618652634530593


#### Задача регрессии

Более лучшие (по MAE, по кросс-валидации и по метрикам после добавления нового признака) для задачи регрессии были показаны моделью Ridge, поэтому улучшенный бейзлайн смормируем на основе модели Ridge с гиперпараметрами:
{'alpha': 0.1, 'solver': 'svd'}

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

ridge = Ridge(max_iter=20000, alpha=0.1, solver='svd')
ridge.fit(X_train_poly_scaled, y_train_new_field)

y_pred_ridge = ridge.predict(X_test_poly_scaled)
ridge_mae = mean_absolute_error(y_test_new_field, y_pred_ridge)
ridge_mse = mean_squared_error(y_test_new_field, y_pred_ridge)

print(f"MAE: {ridge_mae:.2f}")
print(f"MSE: {ridge_mse:.2f}")

MAE: 7.29
MSE: 139.02


### Выводы

В задаче классификации на наборе данных HAR модель логистической регрессии без оптимизаций продемонстрировала хорошие результаты с точностью около 96%. Однако дальнейшая оптимизация с помощью Grid Search и Random Search позволила немного улучшить результаты, достигнув точности в 96.13% и F1-меры 0.96, что подтвердило важность настройки гиперпараметров для улучшения классификации.

В задаче регрессии для прогноза выбросов CO2 модель линейной регрессии без улучшений показала значения MAE и MSE около 13.13 и 377.79 соответственно. После подбора гиперпараметров для Ridge и Lasso моделей ошибки немного снизились, но наибольшее улучшение было достигнуто при добавлении нового признака Engine_Fuel, который позволил существенно уменьшить MAE и MSE, снизив их до 7.29 и 139.02 для Ridge.

Комбинирование настройки гиперпараметров и создания новых признаков значительно улучшило результаты моделей, особенно в задаче регрессии, где добавление информативных признаков дало наибольшее улучшение. Эти подходы подтверждают свою важность для повышения качества моделей в машинном обучении.

## 4. Имплементация алгоритма машинного обучения 

### Имплементация логистической регрессии для классификации

In [ ]:
import numpy as np


class LogisticRegressionCustom:
    def __init__(self, lr=0.01, max_iter=1000, penalty=None, C=1.0, tol=1e-4):
        self.lr = lr
        self.max_iter = max_iter
        self.penalty = penalty
        self.C = C
        self.tol = tol

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X_train, y_train):
        self.X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        self.y_train = y_train
        self.weights = np.zeros(self.X_train.shape[1])
        
        for _ in range(self.max_iter):
            predictions = self._sigmoid(np.dot(self.X_train, self.weights))
            errors = predictions - self.y_train
            gradient = np.dot(self.X_train.T, errors) / len(y_train)
            
            if self.penalty == 'l2':
                gradient += (1 / self.C) * self.weights
            elif self.penalty == 'l1':
                gradient += (1 / self.C) * np.sign(self.weights)
            
            self.weights -= self.lr * gradient
            
            if np.linalg.norm(gradient) < self.tol:
                break

    def predict_proba(self, X_test):
        X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

        return self._sigmoid(np.dot(X_test, self.weights))

    def predict(self, X_test):
        probabilities = self.predict_proba(X_test)

        return (probabilities >= 0.5).astype(int)

### Имплементация линейной регрессии для регрессии

In [ ]:
class LinearRegressionCustom:
    def __init__(self, lr=0.01, max_iter=1000, penalty=None, alpha=0.0, tol=1e-4):
        self.lr = lr
        self.max_iter = max_iter
        self.penalty = penalty
        self.alpha = alpha
        self.tol = tol

    def fit(self, X_train, y_train):
        self.X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
        self.y_train = y_train
        self.weights = np.zeros(self.X_train.shape[1])
        
        for _ in range(self.max_iter):
            predictions = np.dot(self.X_train, self.weights)
            errors = predictions - self.y_train
            gradient = np.dot(self.X_train.T, errors) / len(y_train)
            
            if self.penalty == 'l2':
                gradient += (2 * self.alpha) * self.weights
            elif self.penalty == 'l1':
                gradient += self.alpha * np.sign(self.weights)
            
            self.weights -= self.lr * gradient
            
            if np.linalg.norm(gradient) < self.tol:
                break

    def predict(self, X_test):
        X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
        
        return np.dot(X_test, self.weights)

### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [ ]:
def normalize_data(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

def encode_labels(y):
    label_to_num = {label: idx for idx, label in enumerate(np.unique(y))}
    num_to_label = {idx: label for label, idx in label_to_num.items()}
    y_encoded = np.array([label_to_num[label] for label in y])
    return y_encoded, label_to_num, num_to_label

def train_custom_logistic_regression(train_x, train_y, validation=None):
    train_x_normalized = normalize_data(train_x)
    train_y_encoded, _, _ = encode_labels(train_y)

    if validation is not None:
        validation_x_normalized = normalize_data(validation[0])
        validation_y_encoded, _, _ = encode_labels(validation[1])
    else:
        validation_x_normalized = None
        validation_y_encoded = None

    model = LogisticRegressionCustom(lr=0.01, max_iter=2000, penalty='l1')
    model.fit(train_x_normalized, train_y_encoded)

    y_hat = model.predict(validation_x_normalized)
    acc = metrics.accuracy_score(validation_y_encoded, y_hat)
    print(f"Accuracy = {acc}")
    recall = metrics.recall_score(validation_y_encoded, y_hat, average='weighted')
    precision = metrics.precision_score(validation_y_encoded, y_hat, average='weighted')
    f1 = metrics.f1_score(validation_y_encoded, y_hat, average='weighted')
    print(f"Recall = {recall}")
    print(f"Precision = {precision}")
    print(f"F1 Score = {f1}")

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_custom_logistic_regression(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.18900576857821513
Recall = 0.18900576857821513
Precision = 0.07599169489560975
F1 Score = 0.10811883566310325


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Задача регрессии

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values.astype(np.float64)
y = data['CO2 Emissions(g/km)'].values.astype(np.float64)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

custom_linear_regressor = LinearRegressionCustom(lr=0.01, max_iter=1000)
custom_linear_regressor.fit(X_train, y_train)

predictions = custom_linear_regressor.predict(X_test)

mae = np.mean(np.abs(predictions - y_test))
mse = np.mean((predictions - y_test) ** 2)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


MAE: 13.18
MSE: 384.77


### Сравнение результатов с п.2. Выводы

Результаты сравнения моделей показывают, что самописные реализации логистической и линейной регрессий уступают по качеству моделям из библиотеки sklearn.

В задаче классификации модели из sklearn продемонстрировали высокую точность (Accuracy = 95.86%), а также сбалансированные значения полноты (Recall = 96.17%), точности (Precision = 95.97%) и F1-метрики (F1 Score = 95.84%). В то же время, самописная модель показала крайне низкие результаты: точность составила всего 18.29%, а значения других метрик, таких как полнота, точность и F1, также оказались крайне низкими, что указывает на ее неспособность правильно классифицировать данные. Это может быть связано с проблемой в датесете.

В задаче регрессии разрыв между моделями менее выражен, однако модели из sklearn также показали лучшее качество. Средняя абсолютная ошибка (MAE) для sklearn составила 13.13, а среднеквадратичная ошибка (MSE) — 377.79, что немного лучше результатов самописной модели (MAE = 13.18, MSE = 384.77). Это указывает на то, что самописная линейная регрессия работает корректно, но все же не достигает точности и оптимальности, предложенной готовыми библиотечными реализациями.

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

model = LinearRegressionCustom(lr=0.01, max_iter=2000)

model.fit(X_train_poly_scaled, y_train_new_field)

y_pred_test = model.predict(X_test_poly_scaled)

test_mae = np.mean(np.abs(y_pred_test - y_test_new_field))
test_mse = np.mean((y_pred_test - y_test_new_field) ** 2)

print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


MAE: 9.93
MSE: 247.94


### Подбор гиперпараметров и Grid Search

#### Линейная регрессия

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values.astype(np.float64)
y = data['CO2 Emissions(g/km)'].values.astype(np.float64)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid_linear = {
    'lr': [0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 5000, 10000]
}

best_params = None
best_score = float('inf')
results = []

for lr in param_grid_linear['lr']:
    for max_iter in param_grid_linear['max_iter']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        model = LinearRegressionCustom(lr=lr, max_iter=max_iter)
        model.fit(X_train, y_train)
        
        y_pred_val = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred_val)
        
        results.append({'lr': lr, 'max_iter': max_iter, 'mse': mse})
        
        if mse < best_score:
            best_score = mse
            best_params = {'lr': lr, 'max_iter': max_iter}

best_linear_model = LinearRegressionCustom(lr=best_params['lr'], max_iter=best_params['max_iter'])
best_linear_model.fit(X_train, y_train)

y_pred_test = best_linear_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Лучшие параметры для LinearRegressionCustom:", best_params)
print("Лучший результат на валидационной выборке (MSE):", best_score)
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Лучшие параметры для LinearRegressionCustom: {'lr': 0.1, 'max_iter': 10000}
Лучший результат на валидационной выборке (MSE): 377.9191266873952
MAE: 13.14
MSE: 377.92


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def normalize_data(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

def encode_labels(y):
    label_to_num = {label: idx for idx, label in enumerate(np.unique(y))}
    num_to_label = {idx: label for label, idx in label_to_num.items()}
    y_encoded = np.array([label_to_num[label] for label in y])
    return y_encoded, label_to_num, num_to_label

def read_data(filepath):
    import pandas as pd
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return X, y

def optimize_hyperparameters(train_X, train_y, test_X, test_y):
    train_X_normalized = normalize_data(train_X)
    train_y_encoded, _, _ = encode_labels(train_y)
    test_X_normalized = normalize_data(test_X)
    test_y_encoded, _, _ = encode_labels(test_y)

    param_grid = {
        'lr': [0.001, 0.01, 0.1, 0.5],
        'max_iter': [500, 1000, 2000],
        'penalty': [None, 'l1', 'l2'],
        'C': [0.1, 1.0, 10.0]
    }

    best_params = None
    best_score = 0
    results = []

    for lr in param_grid['lr']:
        for max_iter in param_grid['max_iter']:
            for penalty in param_grid['penalty']:
                for C in param_grid['C']:
                    if penalty is None and C != 1.0:
                        continue

                    model = LogisticRegressionCustom(lr=lr, max_iter=max_iter, penalty=penalty, C=C)
                    model.fit(train_X_normalized, train_y_encoded)

                    y_hat = model.predict(test_X_normalized)
                    acc = accuracy_score(test_y_encoded, y_hat)
                    f1 = f1_score(test_y_encoded, y_hat, average='weighted')

                    results.append({'lr': lr, 'max_iter': max_iter, 'penalty': penalty, 'C': C, 'accuracy': acc, 'f1': f1})

                    if f1 > best_score:
                        best_score = f1
                        best_params = {'lr': lr, 'max_iter': max_iter, 'penalty': penalty, 'C': C}

    best_model = LogisticRegressionCustom(
        lr=best_params['lr'],
        max_iter=best_params['max_iter'],
        penalty=best_params['penalty'],
        C=best_params['C']
    )
    best_model.fit(train_X_normalized, train_y_encoded)
    y_pred = best_model.predict(test_X_normalized)

    acc = accuracy_score(test_y_encoded, y_pred)
    recall = recall_score(test_y_encoded, y_pred, average='weighted')
    precision = precision_score(test_y_encoded, y_pred, average='weighted')
    f1 = f1_score(test_y_encoded, y_pred, average='weighted')

    print("Лучшие параметры:", best_params)
    print(f"Accuracy: {acc:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return best_params, results

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

best_params, results = optimize_hyperparameters(train_X, train_y, test_X, test_y)

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/3672269993.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_1182/

Лучшие параметры: {'lr': 0.5, 'max_iter': 500, 'penalty': 'l1', 'C': 1.0}
Accuracy: 0.1965
Recall: 0.1965
Precision: 0.0831
F1 Score: 0.1160


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации:
{'lr': 0.5, 'max_iter': 500, 'penalty': 'l1', 'C': 1.0}

In [ ]:
def normalize_data(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

def encode_labels(y):
    label_to_num = {label: idx for idx, label in enumerate(np.unique(y))}
    num_to_label = {idx: label for label, idx in label_to_num.items()}
    y_encoded = np.array([label_to_num[label] for label in y])
    return y_encoded, label_to_num, num_to_label

def train_custom_logistic_regression(train_x, train_y, validation=None):
    train_x_normalized = normalize_data(train_x)
    train_y_encoded, _, _ = encode_labels(train_y)

    if validation is not None:
        validation_x_normalized = normalize_data(validation[0])
        validation_y_encoded, _, _ = encode_labels(validation[1])
    else:
        validation_x_normalized = None
        validation_y_encoded = None

    model = LogisticRegressionCustom(lr=0.5, max_iter=500, penalty='l1', C = 1.0)
    model.fit(train_x_normalized, train_y_encoded)

    y_hat = model.predict(validation_x_normalized)
    acc = metrics.accuracy_score(validation_y_encoded, y_hat)
    print(f"Accuracy = {acc}")
    recall = metrics.recall_score(validation_y_encoded, y_hat, average='weighted')
    precision = metrics.precision_score(validation_y_encoded, y_hat, average='weighted')
    f1 = metrics.f1_score(validation_y_encoded, y_hat, average='weighted')
    print(f"Recall = {recall}")
    print(f"Precision = {precision}")
    print(f"F1 Score = {f1}")

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_custom_logistic_regression(train_X, train_y, validation=(test_X, test_y))

Accuracy = 0.19579233118425518
Recall = 0.19579233118425518
Precision = 0.08286520400450481
F1 Score = 0.11559601211525375


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Задача регрессии

Лучшие параметры для регрессии:
{'lr': 0.1, 'max_iter': 10000}

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

model = LinearRegressionCustom(lr=0.01, max_iter=10000)

model.fit(X_train_poly_scaled, y_train_new_field)

y_pred_test = model.predict(X_test_poly_scaled)

test_mae = np.mean(np.abs(y_pred_test - y_test_new_field))
test_mse = np.mean((y_pred_test - y_test_new_field) ** 2)

print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


MAE: 7.99
MSE: 189.17


### Выводы

Сравнение результатов самописных моделей и моделей из библиотеки sklearn показывает значительное преимущество последних. Для задачи классификации самописная логистическая регрессия, даже после оптимизации гиперпараметров с использованием GridSearch, показала низкие результаты: Accuracy составила всего 0.1965, а F1 Score — 0.1160. В то же время логистическая регрессия из sklearn достигла гораздо более высоких значений. Это можно объяснить отсутствием каких-либо оптимизаций в самописной модели логистической регрессии или отсутствием какой-либо предобработки данных, которая есть в моделе в sklearn.

В задаче регрессии самописная линейная регрессия также отстает. После оптимизации гиперпараметров её лучшие показатели составили MAE 7.99 и MSE 189.17, тогда как линейная регрессия из sklearn с добавлением нового признака достигла результатов MAE 7.29 и MSE 139.02.

Таким образом, модели из sklearn демонстрируют значительно более высокую точность по сравнению с самописными решениями, благодаря их оптимизациям.

# Лабораторная работа 3, студент Устинов Денис Александрович М8О-406Б-21

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели с решающим деревом из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

dt_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=42)

dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.8429
Recall: 0.8429
Precision: 0.8434
F1 Score: 0.8415


### b. Обучить модели из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt_regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=None, random_state=42)

dt_regressor.fit(X_train, y_train)

y_pred_test = dt_regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")

Результаты для задачи регрессии:
MAE: 2.80
MSE: 107.49


## 3. Улучшение бейзлайна

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - Для классификации:
        - criterion: выбор функции для оценки разбиений (gini, entropy).
        - max_depth: ограничение на глубину дерева.
        - min_samples_split: минимальное количество выборок для разбиения узла.
        - min_samples_leaf: минимальное количество выборок в листовом узле.
        - max_features: максимальное количество признаков, используемых для разбиения.
    - Для регрессии:
        - criterion: функция оценки качества разбиений (squared_error, absolute_error, poisson).
        - Остальные параметры аналогичны классификации.

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

param_grid_classification = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search_classifier = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_classification,
                                      scoring='f1_weighted', cv=5, verbose=2, n_jobs=-1)

grid_search_classifier.fit(train_X, train_y)

best_params_classifier = grid_search_classifier.best_params_
print("Лучшие параметры для классификации:", best_params_classifier)

best_classifier = grid_search_classifier.best_estimator_
y_pred = best_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для классификации с оптимальными параметрами:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.5s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   3.6s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.9s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.8s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   3.8s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.9s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   4.1s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_spli

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)
X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values.astype(np.float64)
y = data['CO2 Emissions(g/km)'].values.astype(np.float64)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid_regression = {
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

dt_regressor = DecisionTreeRegressor(random_state=42)
grid_search_regressor = GridSearchCV(estimator=dt_regressor, param_grid=param_grid_regression,
                                     scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=-1)

grid_search_regressor.fit(train_X, train_y)

best_params_regressor = grid_search_regressor.best_params_
print("Лучшие параметры для регрессии:", best_params_regressor)

best_regressor = grid_search_regressor.best_estimator_
y_pred = best_regressor.predict(test_X)

mae = mean_absolute_error(test_y, y_pred)
mse = mean_squared_error(test_y, y_pred)

print("Результаты для регрессии с оптимальными параметрами:")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")


Fitting 5 folds for each of 405 candidates, totalling 2025 fits
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error,

#### Добавление нового признака (для задачи регрессии)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=None, random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.52
MSE: 79.11


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

dt_classifier = DecisionTreeClassifier(criterion='gini', max_depth=None,max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42)

dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.8626
Recall: 0.8626
Precision: 0.8636
F1 Score: 0.8620


#### Задача регрессии

Лучшие параметры для регрессии: {'criterion': 'squared_error', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=20, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.43
MSE: 64.43


### Выводы

После проведения оптимизации гиперпараметров и использования метода Grid Search для классификации, удалось добиться небольшого улучшения качества модели: метрика F1 Score выросли с 0.8415 до 0.8620, что свидетельствует о лучшей способности модели корректно классифицировать данные. Для задачи регрессии оптимизация гиперпараметров также привела к улучшению: метрика MAE снизилась с 2.80 до 2.43, а MSE — со 107.49 до 64.63, что указывает на уменьшение средних и квадратичных ошибок модели. 

Это подтверждает важность как правильной настройки модели, так и обогащения исходных данных.

## 4. Имплементация алгоритма машинного обучения 

### Базовый класс

In [ ]:
class DecisionTreeBase:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None

    def _best_split(self, X, y):
        raise NotImplementedError("Этот метод должен быть реализован в дочернем классе.")

    def _build_tree(self, X, y, depth=0):
        if (self.max_depth is not None and depth >= self.max_depth) or len(y) < self.min_samples_split:
            return self._leaf_value(y)

        split = self._best_split(X, y)

        if split is None:
            return self._leaf_value(y)

        feature_index, threshold, left_mask, right_mask = split
        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]

        left_tree = self._build_tree(left_X, left_y, depth + 1)
        right_tree = self._build_tree(right_X, right_y, depth + 1)

        return {
            'feature_index': feature_index,
            'threshold': threshold,
            'left': left_tree,
            'right': right_tree
        }

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, x, tree):
        if not isinstance(tree, dict):
            return tree

        feature_value = x[tree['feature_index']]
        if feature_value <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_sample(row, self.tree) for row in X])


### Имплементация дерева решений для классификации

In [ ]:
class DecisionTreeClassifierCustom(DecisionTreeBase):
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)

    def _gini_impurity(self, left_y, right_y):
        def gini(y):
            m = len(y)

            if m == 0:
                return 0
            
            p = np.bincount(y) / m

            return 1 - np.sum(p ** 2)

        total = len(left_y) + len(right_y)

        return (len(left_y) / total) * gini(left_y) + (len(right_y) / total) * gini(right_y)

    def _leaf_value(self, y):
        return np.bincount(y).argmax()

    def _best_split(self, X, y):
        best_gini = float('inf')
        best_split = None
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                if len(left_y) >= self.min_samples_leaf and len(right_y) >= self.min_samples_leaf:
                    gini = self._gini_impurity(left_y, right_y)
                    if gini < best_gini:
                        best_gini = gini
                        best_split = (feature_index, threshold, left_mask, right_mask)

        return best_split

### Имплементация дерева решений для регрессии

In [ ]:
class DecisionTreeRegressorCustom(DecisionTreeBase):
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)

    def _variance_reduction(self, left_y, right_y):
        def variance(y):
            return np.var(y) * len(y)

        total_len = len(left_y) + len(right_y)
        left_weight = len(left_y) / total_len
        right_weight = len(right_y) / total_len

        return variance(left_y) + variance(right_y)

    def _leaf_value(self, y):
        return np.mean(y)

    def _best_split(self, X, y):
        best_reduction = -float('inf')
        best_split = None
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                if len(left_y) >= self.min_samples_leaf and len(right_y) >= self.min_samples_leaf:
                    reduction = self._variance_reduction(left_y, right_y)
                    if reduction > best_reduction:
                        best_reduction = reduction
                        best_split = (feature_index, threshold, left_mask, right_mask)

        return best_split


### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def read_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_X = train_X.iloc[:800]
train_y = train_y.iloc[:800]

train_sample_indices = np.random.choice(train_X.index, size=800, replace=False)
train_X = train_X.loc[train_sample_indices].reset_index(drop=True)
train_y = train_y.loc[train_sample_indices].reset_index(drop=True)

label_encoder = LabelEncoder()
train_y = pd.Series(label_encoder.fit_transform(train_y), name=train_y.name)
test_y = pd.Series(label_encoder.transform(test_y), name=test_y.name)

dt_classifier = DecisionTreeClassifierCustom(max_depth=4, min_samples_split=2, min_samples_leaf=1)
dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Классификация:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Классификация:
Accuracy: 0.8575
Recall: 0.8575
Precision: 0.7821
F1 Score: 0.8082


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Задача регрессии

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt_regressor = DecisionTreeRegressorCustom(max_depth=300)
dt_regressor.fit(X_train, y_train)
y_pred = dt_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Регрессия:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


Регрессия:
MAE: 7.05
MSE: 251.37


### Сравнение результатов с п.2. Выводы

Кастомная модель показала хороший результат, но проигрывает модели sklearn по всем метрикам, особенно в показателе Precision (0.7821 против 0.8636). Это может свидетельствовать о том, что кастомная модель допускает больше ложных срабатываний (false positives), в то время как модель sklearn более сбалансированно обрабатывает данные.

Разница в метриках между кастомной и библиотечной моделями для регрессии оказалась значительно больше, чем в задаче классификации. Модель sklearn существенно точнее, демонстрируя минимальные ошибки (MAE и MSE). Это говорит о том, что кастомная модель пока плохо справляется с задачей регрессии и требует оптимизации гиперпараметров. 

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressorCustom(max_depth=300)
dt_regressor.fit(X_train, y_train)
y_pred = dt_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Регрессия:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

Регрессия:
MAE: 7.05
MSE: 251.37


#### Оптимизация гиперпараметров и Grid Search

##### Задача классификации

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from itertools import product

def read_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

def grid_search_classification(model_class, param_grid, X, y, metrics):
    best_score = -float('inf')
    best_params = None
    best_metrics = None
    
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    for params in param_combinations:
        model = model_class(**params)
        scores = []
        fold_metrics = []
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            fold_metrics.append({name: metric(y_val, y_pred) for name, metric in metrics.items()})
            scores.append(metrics['accuracy'](y_val, y_pred))
        
        avg_score = np.mean(scores)
        avg_metrics = {name: np.mean([fold[name] for fold in fold_metrics]) for name in metrics}
        
        if avg_score > best_score:
            best_score = avg_score
            best_params = params
            best_metrics = avg_metrics
    
    return best_params, best_metrics

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_X = train_X.iloc[:800]
train_y = train_y.iloc[:800]

train_sample_indices = np.random.choice(train_X.index, size=800, replace=False)
train_X = train_X.loc[train_sample_indices].reset_index(drop=True)
train_y = train_y.loc[train_sample_indices].reset_index(drop=True)

label_encoder = LabelEncoder()
train_y = pd.Series(label_encoder.fit_transform(train_y), name=train_y.name)
test_y = pd.Series(label_encoder.transform(test_y), name=test_y.name)

param_grid = {
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

metrics = {
    'accuracy': accuracy_score,
    'recall': lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted'),
    'precision': lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted'),
    'f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')
}

best_params, best_metrics = grid_search_classification(
    model_class=DecisionTreeClassifierCustom,
    param_grid=param_grid,
    X=train_X,
    y=train_y,
    metrics=metrics
)

print("Лучшие параметры для классификации:", best_params)
print("Средние метрики на кросс-валидации:", best_metrics)

dt_classifier = DecisionTreeClassifierCustom(**best_params)
dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)
test_metrics = {name: metric(test_y, y_pred) for name, metric in metrics.items()}

print("Тестовые метрики:")
for name, value in test_metrics.items():
    print(f"{name.capitalize()}: {value:.4f}")


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

Лучшие параметры для классификации: {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1}
Средние метрики на кросс-валидации: {'accuracy': np.float64(0.9949999999999999), 'recall': np.float64(0.9949999999999999), 'precision': np.float64(0.9952165120507512), 'f1': np.float64(0.9949976722623781)}
Тестовые метрики:
Accuracy: 0.9104
Recall: 0.9104
Precision: 0.9136
F1: 0.9102


##### Задача регрессии

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

def grid_search_regression(model_class, param_grid, X, y, metrics):
    best_score = float('inf')
    best_params = None
    best_metrics = None
    
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    for params in param_combinations:
        model = model_class(**params)
        scores = []
        fold_metrics = []
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            fold_metrics.append({name: metric(y_val, y_pred) for name, metric in metrics.items()})
            scores.append(metrics['mse'](y_val, y_pred))
        
        avg_score = np.mean(scores)
        avg_metrics = {name: np.mean([fold[name] for fold in fold_metrics]) for name in metrics}
        
        if avg_score < best_score:
            best_score = avg_score
            best_params = params
            best_metrics = avg_metrics
    
    return best_params, best_metrics

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")
data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = {
    'max_depth': [100, 300, 500],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

metrics = {
    'mae': mean_absolute_error,
    'mse': mean_squared_error
}

best_params, best_metrics = grid_search_regression(
    model_class=DecisionTreeRegressorCustom,
    param_grid=param_grid,
    X=X_train,
    y=y_train,
    metrics=metrics
)

print("Лучшие параметры для регрессии:", best_params)
print("Средние метрики на кросс-валидации:", best_metrics)

dt_regressor = DecisionTreeRegressorCustom(**best_params)
dt_regressor.fit(X_train, y_train)

y_pred = dt_regressor.predict(X_test)
test_metrics = {name: metric(y_test, y_pred) for name, metric in metrics.items()}

print("Тестовые метрики:")
for name, value in test_metrics.items():
    print(f"{name.upper()}: {value:.2f}")


Лучшие параметры для регрессии: {'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1}
Средние метрики на кросс-валидации: {'mae': np.float64(4.521109864894752), 'mse': np.float64(152.73647327842505)}
Тестовые метрики:
MAE: 4.16
MSE: 172.70


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def read_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_X = train_X.iloc[:800]
train_y = train_y.iloc[:800]

train_sample_indices = np.random.choice(train_X.index, size=800, replace=False)
train_X = train_X.loc[train_sample_indices].reset_index(drop=True)
train_y = train_y.loc[train_sample_indices].reset_index(drop=True)

label_encoder = LabelEncoder()
train_y = pd.Series(label_encoder.fit_transform(train_y), name=train_y.name)
test_y = pd.Series(label_encoder.transform(test_y), name=test_y.name)

dt_classifier = DecisionTreeClassifierCustom(max_depth=5, min_samples_split=2, min_samples_leaf=1)
dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Классификация:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

Классификация:
Accuracy: 0.9104
Recall: 0.9104
Precision: 0.9136
F1 Score: 0.9102


#### Задача регрессии

Лучшие параметры для регрессии: {'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressorCustom(max_depth=500, min_samples_leaf=1, min_samples_split=2)
dt_regressor.fit(X_train, y_train)
y_pred = dt_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Регрессия:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

Регрессия:
MAE: 4.16
MSE: 172.70


### Выводы

После проведения улучшений в бейзлайне для самописных моделей удалось значительно повысить их качество, особенно для задачи классификации. На начальном этапе базовые метрики классификатора показывали достаточно удовлетворительные результаты, такие как Accuracy, Recall и F1 Score на уровне 0.8568, однако Precision составил всего 0.7820, что указывало на значительное количество ложноположительных предсказаний. В задаче регрессии изначальные значения ошибок MAE (45.47) и MSE (3265.88) свидетельствовали о высокой погрешности модели, что говорило о необходимости доработки.

Добавление нового признака для задачи регрессии не повлияло на качество модели, что подтвердили неизменные значения MAE и MSE. Это указывает на то, что данный признак не нес дополнительной информации для улучшения предсказаний. Однако значительное улучшение качества моделей было достигнуто на этапе подбора гиперпараметров с использованием Grid Search. Для задачи классификации удалось добиться хороших: Accuracy, Recall, Precision и F1 Score достигли значений около 0.91. В задаче регрессии также наблюдалось значительное улучшение: MAE снизился до 4.16, а MSE до 172.70, что свидетельствует о повышении точности предсказаний модели.

Если сравнивать метрики самописных моделей и моделей из sklearn, то значительное увеличение точности было достигнуто на задаче классификации, по сравнению с моделью из sklearn. В задаче регрессии самописная модель показала метрики хуже, чем у модели из sklearn.

Таким образом, оптимизация гиперпараметров оказалась наиболее значимым шагом в улучшении качества моделей, что подтверждается резким ростом метрик классификации и заметным снижением ошибок в регрессии.

# Лабораторная работа 4, студент Устинов Денис Александрович М8О-406Б-21

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели со случайным лесом из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

classifier = RandomForestClassifier()

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9250
Recall: 0.9250
Precision: 0.9261
F1 Score: 0.9248


### b. Обучить модели со случайным лесом из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = RandomForestRegressor()

regressor.fit(X_train, y_train)

y_pred_test = regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")

Результаты для задачи регрессии:
MAE: 3.00
MSE: 69.60


## 3. Улучшение бейзлайна

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - Количество деревьев (n_estimators): Определяет число деревьев в лесу. Большее число деревьев увеличивает стабильность модели, но замедляет обучение.
    - Максимальная глубина деревьев (max_depth): Контролирует максимальную глубину каждого дерева. Меньшая глубина снижает переобучение, но может ухудшить качество.
    - Минимальное количество образцов для разделения (min_samples_split): Минимальное количество данных, необходимое для разделения узла.
    - Минимальное количество образцов в листе (min_samples_leaf): Минимальное количество данных в каждом листе дерева.
    - Критерий разбиения (criterion): Для классификации может быть gini или entropy, а для регрессии — squared_error (по умолчанию) или absolute_error.
    - Максимальное количество признаков для выбора (max_features): Указывает, сколько признаков будет использоваться при построении каждого дерева. Значение может быть числовым, дробным (например, 0.5 — половина признаков), sqrt или log2.

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

### Подбор гиперпараметров и Grid Search

#### Задача классификации

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

classifier = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(train_X, train_y)

best_classifier = grid_search.best_estimator_

y_pred = best_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации после оптимизации:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.6s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   3.6s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   3.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   3.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_e

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Результаты для задачи классификации после оптимизации:
Лучшие параметры: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.9342
Recall: 0.9342
Precision: 0.9364
F1 Score: 0.9338


#### Задача регрессии

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

regressor = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    estimator=regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Оптимизация MSE
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

best_regressor = grid_search.best_estimator_

y_pred_test = best_regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии после оптимизации:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_e

/Users/daustinov/study/multimedia/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Результаты для задачи регрессии после оптимизации:
Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
MAE: 2.65
MSE: 35.55


### Формирование новых признаков

#### Задача регрессии

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = RandomForestRegressor(random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.71
MSE: 46.22


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

classifier = RandomForestClassifier(max_depth= 20, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=100)

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9416
Recall: 0.9416
Precision: 0.9431
F1 Score: 0.9411


#### Задача регрессии

Лучшие параметры: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = RandomForestRegressor(max_depth= 20, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 200)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.64
MSE: 33.98


### Выводы

После улучшения базовых моделей для задач классификации и регрессии с использованием Random Forest, удалось достичь заметного повышения качества. В задаче классификации внедрение оптимизации гиперпараметров привело к увеличению метрик точности, полноты, точности предсказания и F1-метрики. Это свидетельствует о повышении способности модели корректно классифицировать данные и снижении ошибок, связанных с неверной категоризацией. В задаче регрессии также удалось улучшить метрики за счет добавления нового признака и последующей оптимизации гиперпараметров. Снижение MAE и MSE указывает на более точное предсказание моделью целевого значения, что отражает ее улучшенную способность улавливать закономерности в данных. Таким образом, проведенные улучшения базовых моделей существенно повысили их производительность и продемонстрировали значимость правильного выбора признаков и настройки гиперпараметров для получения высоких результатов.

## 4. Имплементация алгоритма машинного обучения 

### Базовый класс

In [ ]:
class DecisionTreeCustom:
    def __init__(self, max_depth=None, min_samples_split=2, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.tree = None

    def _gini_impurity(self, left_y, right_y):
        def gini(y):
            m = len(y)
            if m == 0:
                return 0
            p = np.bincount(y) / m
            return 1 - np.sum(p ** 2)

        total = len(left_y) + len(right_y)
        return (len(left_y) / total) * gini(left_y) + (len(right_y) / total) * gini(right_y)

    def _leaf_value(self, y):
        return Counter(y).most_common(1)[0][0]

    def _best_split(self, X, y):
        n_samples, n_features = X.shape

        if isinstance(self.max_features, str):
            if self.max_features == "sqrt":
                n_selected_features = int(np.sqrt(n_features))
            elif self.max_features == "log2":
                n_selected_features = int(np.log2(n_features))
            else:
                raise ValueError(f"Unknown max_features value: {self.max_features}")
        elif self.max_features is None:
            n_selected_features = n_features
        else:
            n_selected_features = self.max_features

        features = np.random.choice(n_features, n_selected_features, replace=False)

        best_gini = float("inf")
        best_split = None

        for feature in features:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                if len(left_y) >= self.min_samples_split and len(right_y) >= self.min_samples_split:
                    gini = self._gini_impurity(left_y, right_y)
                    if gini < best_gini:
                        best_gini = gini
                        best_split = (feature, threshold, left_mask, right_mask)

        return best_split

    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth or len(y) < self.min_samples_split or len(set(y)) == 1:
            return self._leaf_value(y)

        split = self._best_split(X, y)
        if split is None:
            return self._leaf_value(y)

        feature, threshold, left_mask, right_mask = split
        left_tree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_tree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {"feature": feature, "threshold": threshold, "left": left_tree, "right": right_tree}

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, x, tree):
        if not isinstance(tree, dict):
            return tree

        feature_value = x[tree["feature"]]
        if feature_value <= tree["threshold"]:
            return self._predict_sample(x, tree["left"])
        else:
            return self._predict_sample(x, tree["right"])

    def predict(self, X):
        return np.array([self._predict_sample(row, self.tree) for row in X])


### Имплементация случайного леса для классификации

In [ ]:
class RandomForestClassifierCustom:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        if self.random_state:
            np.random.seed(self.random_state)

        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        majority_votes = np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=tree_preds)
        return majority_votes

### Имплементация случайного леса для регрессии

In [ ]:
class RandomForestRegressorCustom:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features=None, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        if self.random_state:
            np.random.seed(self.random_state)

        self.trees = []
        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree = DecisionTreeCustom(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features,
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_preds, axis=0)


### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

rf_classifier = RandomForestClassifierCustom(
    n_estimators=50, max_depth=10, min_samples_split=5, max_features="sqrt", random_state=42
)

rf_classifier.fit(train_X, train_y)
y_pred = rf_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")


Accuracy: 0.9169, Recall: 0.9169, Precision: 0.9193, F1 Score: 0.9166


#### Задача регрессии

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values
y = data['CO2 Emissions(g/km)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_regressor = RandomForestRegressorCustom(
    n_estimators=50, max_depth=10, min_samples_split=5, max_features=5, random_state=42
)
rf_regressor.fit(X_train, y_train)
y_pred_test = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")


MAE: 7.30, MSE: 238.79


### Сравнение результатов с п.2. Выводы

Сравнение результатов обучения моделей из библиотеки sklearn и самописных реализаций для задач классификации и регрессии показало, что модели из sklearn демонстрируют более высокую точность и лучшие метрики качества. В задаче классификации sklearn-модель достигла значения Accuracy 0.9250 и F1 Score 0.9248, тогда как самописная модель показала Accuracy 0.9169 и F1 Score 0.9166. Несмотря на то, что разница в результатах невелика, модели sklearn оказались более устойчивыми в своих предсказаниях, вероятно, благодаря более сложной внутренней реализации и оптимизациям. В задаче регрессии разрыв между результатами оказался более заметным: MAE и MSE для sklearn-модели составили 3.00 и 69.60 соответственно, в то время как для самописной модели эти метрики равнялись 7.30 и 238.79. Это указывает на то, что самописная реализация пока уступает библиотечным вариантам в точности предсказаний, что может быть связано с ограничениями в использовании усреднения, глубины деревьев или других аспектов алгоритма. Таким образом, библиотечные реализации более эффективны и удобны для использования в реальных задачах, но создание собственных моделей позволяет лучше понять принципы работы алгоритмов и их внутренние механизмы.

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

X_train_new_field = X_train_new_field.values
X_test_new_field = X_test_new_field.values
y_train_new_field = y_train_new_field.values
y_test_new_field = y_test_new_field.values

rf_regressor = RandomForestRegressorCustom(
    n_estimators=50, max_depth=10, min_samples_split=5, max_features=5, random_state=42
)
rf_regressor.fit(X_train_new_field, y_train_new_field)
y_pred_test = rf_regressor.predict(X_test_new_field)

mae = mean_absolute_error(y_test_new_field, y_pred_test)
mse = mean_squared_error(y_test_new_field, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 6.75, MSE: 207.63


#### Оптимизация гиперпараметров и Grid Search

##### Задача классификации

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def grid_search_rf_classification(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = -np.inf

    for n_estimators in param_grid['n_estimators']:
        for min_samples_split in param_grid['min_samples_split']:
            for max_features in param_grid['max_features']:
                rf_classifier = RandomForestClassifierCustom(
                    n_estimators=n_estimators,
                    max_depth=10,
                    min_samples_split=min_samples_split,
                    max_features=max_features,
                    random_state=42
                )
                rf_classifier.fit(X_train, y_train)
                y_pred = rf_classifier.predict(X_test)
                
                accuracy = accuracy_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred, average='weighted')

                if f1 > best_score:
                    best_score = f1
                    best_params = {
                        'n_estimators': n_estimators,
                        'min_samples_split': min_samples_split,
                        'max_features': max_features
                    }

    print(f"Best parameters: {best_params}")
    print(f"Best F1 Score: {best_score:.4f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2']
}

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

best_params_classification = grid_search_rf_classification(train_X, train_y, test_X, test_y, param_grid)


Best parameters: {'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'log2'}
Best F1 Score: 0.9248


##### Задача регрессии

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def grid_search_rf_regression(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = np.inf

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for min_samples_split in param_grid['min_samples_split']:
                for max_features in param_grid['max_features']:
                    rf_regressor = RandomForestRegressorCustom(
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        min_samples_split=min_samples_split,
                        max_features=max_features,
                        random_state=42
                    )
                    rf_regressor.fit(X_train, y_train)
                    y_pred = rf_regressor.predict(X_test)
                    
                    mse = mean_squared_error(y_test, y_pred)

                    if mse < best_score:
                        best_score = mse
                        best_params = {
                            'n_estimators': n_estimators,
                            'max_depth': max_depth,
                            'min_samples_split': min_samples_split,
                            'max_features': max_features
                        }

    print(f"Best parameters: {best_params}")
    print(f"Best MSE: {best_score:.2f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5, 10],
    'max_features': [None, 3, 5]
}

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values
y = data['CO2 Emissions(g/km)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

best_params_regression = grid_search_rf_regression(X_train, y_train, X_test, y_test, param_grid)


Best parameters: {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2, 'max_features': 5}
Best MSE: 202.25


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'n_estimators': 100, 'min_samples_split': 5, 'max_features': 'log2'}

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

rf_classifier = RandomForestClassifierCustom(
    n_estimators=100, max_depth=10, min_samples_split=5, max_features="log2", random_state=42
)

rf_classifier.fit(train_X, train_y)
y_pred = rf_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")

Accuracy: 0.9240, Recall: 0.9240, Precision: 0.9274, F1 Score: 0.9232


#### Задача регрессии

Лучшие параметры для регрессии: {'n_estimators': 50, 'max_depth': 10, 'min_samples_split': 2, 'max_features': 5}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

X_train_new_field = X_train_new_field.values
X_test_new_field = X_test_new_field.values
y_train_new_field = y_train_new_field.values
y_test_new_field = y_test_new_field.values

rf_regressor = RandomForestRegressorCustom(
    n_estimators=50, max_depth=10, min_samples_split=2, max_features=5, random_state=42
)
rf_regressor.fit(X_train, y_train)
y_pred_test = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 6.88, MSE: 202.25


### Выводы

После улучшения бейзлайна модели из sklearn продемонстрировали существенное повышение качества предсказаний как в задаче классификации, так и в задаче регрессии, превосходя показатели самописных реализаций. В классификации после оптимизации гиперпараметров sklearn-модель достигла значений Accuracy 0.9416 и F1 Score 0.9411, что указывает на ее высокую точность и сбалансированность. Самописная модель, хотя и показала улучшенные результаты по сравнению с первоначальными, с Accuracy 0.9240 и F1 Score 0.9232, все же уступила библиотечному аналогу, вероятно, из-за более ограниченных возможностей оптимизации. 

В задаче регрессии модели sklearn вновь оказались значительно эффективнее. После добавления нового признака и подбора гиперпараметров метрики MAE и MSE снизились до 2.64 и 33.98 соответственно, демонстрируя более точные и стабильные предсказания. Самописная модель, хотя и показала улучшение метрик (MAE: 6.88, MSE: 202.25), все же значительно отстала от sklearn-реализации. Это может быть связано с недостаточной реализацией некоторых ключевых аспектов случайного леса, таких как оптимизация количества признаков при разбиении или ограничений на глубину деревьев.

Таким образом, после улучшения бейзлайна модели sklearn продемонстрировали превосходство по всем ключевым метрикам, что делает их более подходящими для использования в практических задачах.

# Лабораторная работа 5, студент Устинов Денис Александрович М8О-406Б-21

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели с градиентным бустингом из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

classifier = GradientBoostingClassifier()

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9237
Recall: 0.9237
Precision: 0.9241
F1 Score: 0.9237


### b. Обучить модели с градиентным бустингом из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = GradientBoostingRegressor()

regressor.fit(X_train, y_train)

y_pred_test = regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")

Результаты для задачи регрессии:
MAE: 5.13
MSE: 94.89


## 3. Улучшение бейзлайна

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - n_estimators: Количество деревьев в ансамбле.
    - max_depth: Максимальная глубина деревьев.
    - learning_rate: Скорость обучения.

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

### Подбор гиперпараметров и Grid Search

#### Задача классификации

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2]
}

classifier = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(train_X, train_y)

print("Лучшие параметры для классификации:")
print(grid_search.best_params_)

best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты после оптимизации гиперпараметров для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time= 1.6min
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time= 1.6min
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time= 1.6min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time= 3.1min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time= 3.1min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time= 3.1min
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time= 2.5min
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time= 2.5min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time= 4.7min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time= 4.7min
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time= 2.5min
[CV] END ...learning_rate=0.1, max_depth=3, n_es

#### Задача регрессии

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2]
}

regressor = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Лучшие параметры для регрессии:")
print(grid_search.best_params_)

best_regressor = grid_search.best_estimator_
y_pred_test = best_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print("Результаты после оптимизации гиперпараметров для задачи регрессии:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   0.1s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   0.1s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   0.1s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.3s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time=   0.3s
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time=   0.2s
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time=   0.4s
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_es

### Формирование новых признаков

#### Задача регрессии

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = GradientBoostingRegressor(random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 4.91
MSE: 82.05


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации:
{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 150}

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

classifier = GradientBoostingClassifier(max_depth= 3, learning_rate = 0.2, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=150)

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9376
Recall: 0.9376
Precision: 0.9388
F1 Score: 0.9374


#### Задача регрессии

Лучшие параметры для регрессии:
{'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = GradientBoostingRegressor(max_depth= 5, learning_rate=0.2, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 150)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 3.20
MSE: 38.18


### Выводы

После проведения оптимизации гиперпараметров для моделей градиентного бустинга из библиотеки sklearn удалось существенно улучшить их результаты как в задаче классификации, так и в задаче регрессии. В задаче классификации наблюдается заметное повышение всех основных метрик: точности, полноты, точности предсказаний и F1-меры. Например, точность модели увеличилась с 0.9237 до 0.9376, что свидетельствует о лучшем распознавании классов. В задаче регрессии также достигнуты значительные улучшения. Средняя абсолютная ошибка (MAE) снизилась с 5.13 до 3.20, а среднеквадратичная ошибка (MSE) уменьшилась с 94.89 до 38.18, что говорит о более точных предсказаниях целевой переменной. Эти результаты подтверждают эффективность подхода по подбору оптимальных гиперпараметров для повышения производительности моделей.

## 4. Имплементация алгоритма машинного обучения 

### Имплементация градиентного бустинга для классификации

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, log_loss

class GradientBoostingClassifierCustom:
    def __init__(self, n_estimators=100, max_depth=3, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.models = []
        self.initial_prediction = None
        self.classes_ = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.classes_ = np.unique(y)
        self.models = {cls: [] for cls in self.classes_}
        self.initial_prediction = {cls: np.log(np.sum(y == cls) / len(y)) for cls in self.classes_}
        y_one_hot = np.array([[1 if yi == cls else 0 for cls in self.classes_] for yi in y])

        for cls_idx, cls in enumerate(self.classes_):
            residuals = y_one_hot[:, cls_idx] - self._sigmoid(self.initial_prediction[cls])
            for _ in range(self.n_estimators):
                tree = DecisionTreeRegressor(max_depth=self.max_depth)
                tree.fit(X, residuals)
                predictions = tree.predict(X)
                residuals -= self.learning_rate * predictions
                self.models[cls].append(tree)

    def predict_proba(self, X):
        X = np.array(X)
        logits = {cls: np.full(X.shape[0], self.initial_prediction[cls]) for cls in self.classes_}
        for cls in self.classes_:
            for tree in self.models[cls]:
                logits[cls] += self.learning_rate * tree.predict(X)

        exp_logits = np.exp(np.array(list(logits.values())).T)
        probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
        return probabilities

    def predict(self, X):
        proba = self.predict_proba(X)
        return np.array(self.classes_)[np.argmax(proba, axis=1)]

    @staticmethod
    def _sigmoid(logits):
        return 1 / (1 + np.exp(-logits))

### Имплементация градиентного бустинга для регрессии

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, log_loss

class GradientBoostingRegressorCustom:
    def __init__(self, n_estimators=100, max_depth=3, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.models = []
        self.initial_prediction = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.models = []
        self.initial_prediction = np.mean(y)
        residuals = y - self.initial_prediction

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            predictions = tree.predict(X)
            residuals -= self.learning_rate * predictions
            self.models.append(tree)

    def predict(self, X):
        X = np.array(X)
        predictions = np.full(X.shape[0], self.initial_prediction)
        for tree in self.models:
            predictions += self.learning_rate * tree.predict(X)
        return predictions


### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [ ]:
from sklearn.preprocessing import LabelEncoder

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

gb_classifier = GradientBoostingClassifierCustom(
    n_estimators=10, max_depth=3, learning_rate=0.1
)
gb_classifier.fit(train_X, train_y)
y_pred = gb_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.8297
Recall: 0.8297
Precision: 0.8540
F1 Score: 0.8271


#### Задача регрессии

In [ ]:
data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

gb_regressor = GradientBoostingRegressorCustom(
    n_estimators=20, max_depth=5, learning_rate=0.1
)
gb_regressor.fit(X_train, y_train)
y_pred_test = gb_regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 8.26
MSE: 163.69


### Сравнение результатов с п.2. Выводы

Сравнение результатов обучения моделей из sklearn и собственных имплементаций демонстрирует существенное превосходство библиотечных решений по всем метрикам для задач классификации и регрессии. Для классификации модели из sklearn обеспечивают значительно более высокие значения Accuracy, Recall, Precision и F1 Score, что указывает на их более точное и устойчивое разделение классов. В то же время самописная модель заметно уступает по точности и качеству предсказаний, хотя её Precision остаётся относительно высоким, что может свидетельствовать о склонности к снижению ложных срабатываний, но в ущерб общему охвату (Recall).  

Для регрессии модели из sklearn также демонстрируют более точные предсказания, обеспечивая значительно меньшие значения MAE и MSE. Это указывает на способность библиотечной реализации более эффективно минимизировать отклонения предсказаний от истинных значений. Самописная модель, напротив, показывает большее среднее абсолютное и квадратичное отклонение, что говорит о её меньшей эффективности в прогнозировании.  

Собственные реализации, хотя и демонстрируют базовую функциональность, существенно уступают в производительности и требуют дальнейшего улучшения для достижения конкурентных результатов.

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

rf_regressor = GradientBoostingRegressorCustom(
    n_estimators=20, max_depth=5, learning_rate=0.1
)
rf_regressor.fit(X_train_new_field, y_train_new_field)
y_pred_test = rf_regressor.predict(X_test_new_field)

mae = mean_absolute_error(y_test_new_field, y_pred_test)
mse = mean_squared_error(y_test_new_field, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 8.15, MSE: 160.08


#### Оптимизация гиперпараметров и Grid Search

##### Задача классификации

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, mean_absolute_error, mean_squared_error

def grid_search_classification(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = -np.inf

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for learning_rate in param_grid['learning_rate']:
                print(f"Testing params: n_estimators={n_estimators}, max_depth={max_depth}, learning_rate={learning_rate}")
                model = GradientBoostingClassifierCustom(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate
                )
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                f1 = f1_score(y_test, y_pred, average='weighted')
                if f1 > best_score:
                    best_score = f1
                    best_params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'learning_rate': learning_rate
                    }

    print(f"Best params for classification: {best_params}")
    print(f"Best F1 Score: {best_score:.4f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

best_params_classification = grid_search_classification(train_X, train_y, test_X, test_y, param_grid)

Testing params: n_estimators=10, max_depth=3, learning_rate=0.01
Testing params: n_estimators=10, max_depth=3, learning_rate=0.1
Testing params: n_estimators=10, max_depth=3, learning_rate=0.2
Testing params: n_estimators=10, max_depth=5, learning_rate=0.01
Testing params: n_estimators=10, max_depth=5, learning_rate=0.1
Testing params: n_estimators=10, max_depth=5, learning_rate=0.2
Testing params: n_estimators=10, max_depth=7, learning_rate=0.01
Testing params: n_estimators=10, max_depth=7, learning_rate=0.1
Testing params: n_estimators=10, max_depth=7, learning_rate=0.2
Testing params: n_estimators=50, max_depth=3, learning_rate=0.01
Testing params: n_estimators=50, max_depth=3, learning_rate=0.1
Testing params: n_estimators=50, max_depth=3, learning_rate=0.2
Testing params: n_estimators=50, max_depth=5, learning_rate=0.01
Testing params: n_estimators=50, max_depth=5, learning_rate=0.1
Testing params: n_estimators=50, max_depth=5, learning_rate=0.2
Testing params: n_estimators=50, ma

##### Задача регрессии

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def grid_search_regression(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = np.inf

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for learning_rate in param_grid['learning_rate']:
                print(f"Testing params: n_estimators={n_estimators}, max_depth={max_depth}, learning_rate={learning_rate}")
                model = GradientBoostingRegressorCustom(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate
                )
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                mse = mean_squared_error(y_test, y_pred)
                if mse < best_score:
                    best_score = mse
                    best_params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'learning_rate': learning_rate
                    }

    print(f"Best params for regression: {best_params}")
    print(f"Best MSE: {best_score:.4f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values
y = data['CO2 Emissions(g/km)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

best_params_regression = grid_search_regression(X_train, y_train, X_test, y_test, param_grid)


Testing params: n_estimators=10, max_depth=3, learning_rate=0.01
Testing params: n_estimators=10, max_depth=3, learning_rate=0.1
Testing params: n_estimators=10, max_depth=3, learning_rate=0.2
Testing params: n_estimators=10, max_depth=5, learning_rate=0.01
Testing params: n_estimators=10, max_depth=5, learning_rate=0.1
Testing params: n_estimators=10, max_depth=5, learning_rate=0.2
Testing params: n_estimators=10, max_depth=7, learning_rate=0.01
Testing params: n_estimators=10, max_depth=7, learning_rate=0.1
Testing params: n_estimators=10, max_depth=7, learning_rate=0.2
Testing params: n_estimators=50, max_depth=3, learning_rate=0.01
Testing params: n_estimators=50, max_depth=3, learning_rate=0.1
Testing params: n_estimators=50, max_depth=3, learning_rate=0.2
Testing params: n_estimators=50, max_depth=5, learning_rate=0.01
Testing params: n_estimators=50, max_depth=5, learning_rate=0.1
Testing params: n_estimators=50, max_depth=5, learning_rate=0.2
Testing params: n_estimators=50, ma

### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.2}

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

rf_classifier = GradientBoostingClassifierCustom(n_estimators=100, max_depth=3, learning_rate=0.2)

rf_classifier.fit(train_X, train_y)
y_pred = rf_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")

Accuracy: 0.9013, Recall: 0.9013, Precision: 0.9089, F1 Score: 0.9007


#### Задача регрессии

Лучшие параметры для регрессии: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.2}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

X_train_new_field = X_train_new_field.values
X_test_new_field = X_test_new_field.values
y_train_new_field = y_train_new_field.values
y_test_new_field = y_test_new_field.values

rf_regressor = GradientBoostingRegressorCustom(n_estimators=100, max_depth=5, learning_rate=0.2)
rf_regressor.fit(X_train, y_train)
y_pred_test = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 3.30, MSE: 52.01


### Выводы

Сравнение метрик улучшенных моделей из sklearn и самописных моделей демонстрирует, что модели из sklearn после оптимизации гиперпараметров показывают более высокую точность и лучшее качество как в задаче классификации, так и в задаче регрессии. 

В задаче классификации улучшенные модели из sklearn достигли Accuracy 93.76%, что превосходит показатель самописной модели с её Accuracy 90.13%. Аналогичная тенденция наблюдается для метрик Recall, Precision и F1 Score, которые для моделей sklearn остаются выше. Это связано с высокой оптимизацией и эффективностью алгоритмов sklearn, а также с их точной настройкой через Grid Search. В свою очередь, самописная модель также показала приемлемое качество, особенно учитывая её простоту и кастомную реализацию.

В задаче регрессии самописная модель продемонстрировала MAE 3.30 и MSE 52.01, что близко к результатам моделей из sklearn (MAE 3.20 и MSE 38.18). Это указывает на то, что собственная реализация алгоритма градиентного бустинга справляется с задачей регрессии достаточно хорошо, хотя модели sklearn по-прежнему остаются более точными благодаря их внутренним оптимизациям и точному управлению регуляризацией.

Таким образом, можно сделать вывод, что модели из sklearn, особенно после оптимизации гиперпараметров, показывают более высокую производительность. Однако результаты самописных моделей свидетельствуют о корректной реализации алгоритма и его способности решать задачи с приемлемой точностью.

# Сравнительная таблица метрик обучения моделей из всех ЛР

| Алгоритм              | Задача          | Бейзлайн                                                                 | Улучшенный бейзлайн                                                   | Самостоятельная имплементация                                       | Улучшенная самостоятельная имплементация                            |
|------------------------|-----------------|-------------------------------------------------------------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| **KNN**               | Классификация  | Accuracy = 0.9002 <br> Recall = 0.9002 <br> Precision = 0.9039 <br> F1 Score = 0.8993 | Accuracy = 0.9134 <br> Recall = 0.9134 <br> Precision = 0.9171 <br> F1 Score = 0.9126 | Accuracy = 0.9002 <br> Recall = 0.9002 <br> Precision = 0.9002 <br> F1 Score = 0.8993 | Accuracy = 0.9141 <br> Recall = 0.9177 <br> Precision = 0.9141 <br> F1 Score = 0.9133 |
|                        | Регрессия      | MAE: 3.93 <br> MSE: 76.53                     | MAE: 2.24 <br> MSE: 34.31                   | MAE: 3.50 <br> MSE: 76.6                  | MAE: 2.32 <br> MSE: 32.90                   |
| **Линейные модели**    | Классификация  | Accuracy = 0.9586 <br> Recall = 0.9586 <br> Precision = 0.9597 <br> F1 Score = 0.9584 | Accuracy = 0.9619 <br> Recall = 0.9619 <br> Precision = 0.9632 <br> F1 Score = 0.9618 | Accuracy = 0.1890 <br> Recall = 0.1890 <br> Precision = 0.0759 <br> F1 Score = 0.1081 | Accuracy = 0.1957 <br> Recall = 0.1957 <br> Precision = 0.0828 <br> F1 Score = 0.1155 |
|                        | Регрессия      | MAE: 13.13 <br> MSE: 377.79                     | MAE: 7.29 <br> MSE: 139.02                   | MAE: 13.18 <br> MSE: 384.77                   | MAE: 7.99 <br> MSE: 189.17                   |
| **Решающее дерево**    | Классификация  | Accuracy = 0.8429 <br> Recall = 0.8429 <br> Precision = 0.8434 <br> F1 Score = 0.8415 | Accuracy = 0.8626 <br> Recall = 0.8626 <br> Precision = 0.8636 <br> F1 Score = 0.8620 | Accuracy = 0.8575 <br> Recall = 0.8575 <br> Precision = 0.7821 <br> F1 Score = 0.8082 | Accuracy = 0.9104 <br> Recall = 0.9104 <br> Precision = 0.9136 <br> F1 Score = 0.9102 |
|                        | Регрессия      | MAE: 2.80 <br> MSE: 107.49                     | MAE: 2.43 <br> MSE: 64.43                   | MAE: 7.05 <br> MSE: 251.37                   | MAE: 4.16 <br> MSE: 172.70                   |
| **Случайный лес**      | Классификация  | Accuracy = 0.9250 <br> Recall = 0.9250 <br> Precision = 0.9261 <br> F1 Score = 0.9248 | Accuracy = 0.9416 <br> Recall = 0.9416 <br> Precision = 0.9431 <br> F1 Score = 0.9411 | Accuracy = 0.9169 <br> Recall = 0.9169 <br> Precision = 0.9193 <br> F1 Score = 0.9166 | Accuracy = 0.9240 <br> Recall = 0.9240 <br> Precision = 0.9274 <br> F1 Score = 0.9232 |
|                        | Регрессия      | MAE: 3.00 <br> MSE: 69.60                     | MAE: 2.64 <br> MSE: 33.98                   | MAE: 7.30 <br> MSE: 238.79                   | MAE: 6.88 <br> MSE: 202.25                   |
| **Градиентный бустинг**| Классификация  | Accuracy = 0.9237 <br> Recall = 0.9237 <br> Precision = 0.9241 <br> F1 Score = 0.9237 | Accuracy = 0.9376 <br> Recall = 0.9376 <br> Precision = 0.9388 <br> F1 Score = 0.9374 | Accuracy = 0.8297 <br> Recall = 0.8297 <br> Precision = 0.8540 <br> F1 Score = 0.8271 | Accuracy = 0.9013 <br> Recall = 0.9013 <br> Precision = 0.9089 <br> F1 Score = 0.9007 |
|                        | Регрессия      | MAE: 5.13 <br> MSE: 94.89                     | MAE: 3.20 <br> MSE: 38.18                   | MAE: 8.26 <br> MSE: 163.69                   | MAE: 3.30 <br> MSE: 52.01                   |